In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv("data/train_data.csv",sep='\t',header=None)
songs = pd.read_csv("data/songs.csv",sep='\t',header=None)
users = pd.read_csv("data/users.csv",sep='\t',header=None)
test_data = pd.read_csv("data/test_data.csv",sep='\t',header=None)

In [3]:
train_data = train_data.rename(columns={train_data.columns[0]:"user",train_data.columns[1]:"song",train_data.columns[2]:"number"})
test_data = test_data.rename(columns={test_data.columns[0]:"user",test_data.columns[1]:"song",test_data.columns[2]:"number"})
songs = songs.rename(columns={songs.columns[0]:"song"})
users = users.rename(columns={users.columns[0]:"user"})


In [4]:
users["user_id"] = ["user_" +str(i) for i in range(5000)]
songs["song_id"] = ["song_" +str(i) for i in range(49564)]

In [46]:
max(train_data.number)

463

In [5]:
def transformation(data, users, songs):
    tmp =pd.merge(data,users,how='left',left_on=data.user, right_on=users.user)
    tmp1 = pd.merge(tmp,songs,how='left',left_on=tmp.song, right_on=songs.song)
    return tmp1[tmp1.columns[[4,6,2]]]

In [6]:
train_data = transformation(train_data,users,songs)
test_data = transformation(test_data, users, songs)

In [7]:
n,m = len(users),len(songs)

In [8]:
matrix = np.zeros((n,m))

In [9]:
def to_matrix(data,matrix):
    for r in train_data.iterrows():
        tmp = r[1]
        row = int(tmp[0].split("_")[-1])
        col = int(tmp[1].split("_")[-1])
        number = tmp[2]
        matrix[row][col] = number
    return matrix

In [10]:
V_train = to_matrix(train_data,matrix)

In [11]:
V_test = to_matrix(test_data, matrix)

In [53]:
x = np.array([[1,2,0],[-1,2,5]])
x[x<1]=2
x

array([[1, 2, 2],
       [2, 2, 5]])

In [12]:
def indicate(matrix):
    index_row,index_col = np.nonzero(matrix)
    M = np.zeros(matrix.shape)
    for i in range(len(index_row)):
        M[index_row[i]][index_col[i]] = 1
    return M

In [54]:
def normalization(matrix, max_rating ):
    matrix[matrix<0] = 0
    matrix[matrix>max_rating] = max_rating
    return matrix

In [57]:
def SVD_incremental(V_train,feature_number,iteration_number,regu = 0.02,max_rating =483, learning_rate = 0.05):
    U = np.ones((n, feature_number))*0.01
    M = np.ones((m, feature_number))*0.01
    I = indicate(V_train)
    for i in range(iteration_number):
        MUL = np.dot(U,M.T)
        MUL = normalization(MUL, max_rating)
        tmp = np.multiply((V_train - MUL),I)
        delta_U = []
        delta_M = []
        for i in range(n):
            delta_U.append(np.sum(tmp[i]*M.T,axis=1))
        delta_U= np.array(delta_U) - regu*U

        for i in range(m):
            delta_M.append(np.sum(tmp.T[i]*U.T,axis=1))
        delta_M= np.array(delta_M) - regu*M
        
        
        U = U + delta_U*learning_rate
        M = M + delta_M *learning_rate
        
    return U,M

In [43]:
kkk = []
for i in range(m):
    kkk.append(np.sum(tmp.T[i]*U.T,axis=1))

In [45]:
kkk = np.array(kkk)
kkk-0.02*M

array([[ 0.019796,  0.019796],
       [-0.0002  , -0.0002  ],
       [-0.0002  , -0.0002  ],
       ..., 
       [-0.0002  , -0.0002  ],
       [-0.0002  , -0.0002  ],
       [ 0.019798,  0.019798]])

In [354]:
tmp = y*x
np.sum(tmp,axis=1)

array([[23, 28, 33],
       [47, 58, 69],
       [ 8, 10, 12]])

In [319]:
np.nonzero(np.multiply(V_train,indicate(V_train))[0])

(array([ 2889,  9733, 11195, 11814, 13995, 14362, 19962, 22433, 23812,
        32134, 33351, 36979]),)